<a href="https://colab.research.google.com/github/CarlosAraico/App-Manolito-/blob/main/backend_main_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sentry_sdk
from fastapi import FastAPI, HTTPException, Depends
from pydantic import BaseModel, Field
from typing import Literal
from fastapi.middleware.cors import CORSMiddleware

# --- 1. Configuración & Observabilidad ---
SENTRY_DSN = os.getenv("SENTRY_DSN")
if SENTRY_DSN:
    sentry_sdk.init(dsn=SENTRY_DSN, traces_sample_rate=1.0)

app = FastAPI(
    title="Don Manolito API",
    description="Microservicio de IA y Gestión para Don Manolito",
    version="1.0.0"
)

# CORS para permitir peticiones desde el Frontend (Next.js)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], # En producción cambiar por el dominio real
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# --- 2. Modelos de Datos (Pydantic) ---
class EventQuoteRequest(BaseModel):
    type: Literal['Cumpleaños', 'Corporativo', 'Boda']
    people: int = Field(..., gt=0, description="Número de personas")
    budget: Literal['Bajo', 'Medio', 'Alto']

class EventQuoteResponse(BaseModel):
    suggestion: str
    estimated_cost: float

# --- 3. Lógica de Negocio (Mock de Gemini) ---
# Aquí conectarías la llamada real a Google Gemini
def generate_ai_quote(data: EventQuoteRequest) -> str:
    # Simulación de respuesta de LLM
    prompts = {
        "Cumpleaños": "¡Fiesta inolvidable! Tacos al pastor y cervezas frías.",
        "Corporativo": "Menú ejecutivo: Arrachera premium y aguas frescas.",
        "Boda": "Banquete de lujo: Rib Eye, Costra de queso y vino tinto."
    }
    base_text = prompts.get(data.type, "Evento especial")
    return f"{base_text} Para {data.people} pax. Presupuesto {data.budget}."

# --- 4. Endpoints ---
@app.get("/health")
def health_check():
    return {"status": "ok", "db": "connected"}

@app.post("/quote", response_model=EventQuoteResponse, summary="Generar cotización con IA")
async def generate_quote(request: EventQuoteRequest):
    try:
        # Aquí iría la llamada a services.gemini_service
        suggestion = generate_ai_quote(request)

        # Calculo simple para demo
        base_price = 150 if request.budget == "Bajo" else 250 if request.budget == "Medio" else 450
        total = base_price * request.people

        return {
            "suggestion": suggestion,
            "estimated_cost": total
        }
    except Exception as e:
        # Sentry capturará esto automáticamente
        raise HTTPException(status_code=500, detail=str(e))